# Loading Image

We use this section to load an image and preprocess it. 

In [457]:
# Imports for visualization
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

In [458]:
def display_image(img):
    #imS = cv2.resize(img, (1000, 700))  # Resize image
    cv2.imshow("image", img)
    cv2.waitKey(0)

    cv2.waitKey(0)
    cv2.destroyAllWindows()
    

def make_kernel(a):
  """Transform a 2D array into a convolution kernel"""
  a = np.asarray(a)
  a = a.reshape(list(a.shape) + [1,1])
  return a

In [459]:
#Load image and convert to grayscale
image_path = 'test_11.jpg'
img = cv2.imread(image_path)
img = cv2.resize(img, (800, 800))
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

#Perform Hough transform
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
edges = cv2.Canny(gray,50,150,apertureSize = 3)
lines = cv2.HoughLines(edges,1,np.pi/180,200)



In [460]:
vertical_lines = []
horizontal_lines = []
#Display all lines
for line in lines:
    rho = line[0][0]
    theta = line[0][1]
    if(theta == 0):
        vertical_lines.append(line)



#Display all lines
for line in lines:
    rho = line[0][0]
    theta = line[0][1]
    if theta != 0.0:        horizontal_lines.append(line)
        

In [461]:
def checkMatch(lineset):
    """Checks whether there exists 7 lines of consistent increasing order in set of lines"""
    linediff = np.diff(lineset)
    x = 0
    cnt = 0
    for line in linediff:
        # Within 5 px of the other (allowing for minor image errors)
        if np.abs(line - x) < 5:
            cnt += 1
        else:
            cnt = 0
            x = line
    return cnt == 5

In [462]:
len(horizontal_lines)

21

In [463]:
img = cv2.imread(image_path)
img = cv2.resize(img, (1000, 1000))
for line in vertical_lines:
    rho = line[0][0]
    theta = line[0][1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

display_image(img)

In [448]:

#Convert to numpy arrays. 
horizontal_lines_np = np.concatenate( horizontal_lines, axis=0 )
vertical_lines_np = np.concatenate( vertical_lines, axis=0 )
horizontal_lines_np = np.sort(horizontal_lines_np, axis = 0)
vertical_lines_np = np.sort(vertical_lines_np, axis = 0)

horizontal_lines_np

array([[118.       ,   1.5707964]], dtype=float32)

In [449]:
linediff_hor = np.diff(horizontal_lines_np, axis = 0)
linediff_ver = np.diff(vertical_lines_np, axis = 0)

linediff_hor

array([], shape=(0, 2), dtype=float32)

In [450]:
#Remove lines which are very close to each other so that the board detection will be easier. 
def filter_lines(lines, linediff):
    indices_to_delete = []
    for index, diff in enumerate(linediff): 
        if(diff[0] <= 10):
            indices_to_delete.append(index)
    return np.delete(lines, indices_to_delete,0)


In [451]:
filtered_horizontal_lines = filter_lines(horizontal_lines_np, linediff_hor)
filtered_vertical_lines = filter_lines(vertical_lines_np, linediff_ver)

In [452]:
img = cv2.imread(image_path)
img = cv2.resize(img, (1000, 1000))
for line in filtered_horizontal_lines:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

display_image(img)

In [453]:
#update line differences. 

linediff_hor = np.diff(filtered_horizontal_lines, axis = 0)
linediff_ver = np.diff(filtered_vertical_lines, axis = 0)

In [454]:
linediff_hor

array([], shape=(0, 2), dtype=float32)

In [455]:
def get_chess_lines(lines, linediff):
    indices_to_delete = []
    #find most occuring difference which is probably the width of the chess square. 
    m = stats.mode(linediff)
    mode = m[0][0][0]
    for index, diff in enumerate(linediff): 
        if(abs(diff[0] - mode) >5):
            print(diff[0])
            indices_to_delete.append(index+1)
    return np.delete(lines, indices_to_delete,0)

In [456]:
chess_lines_hor = get_chess_lines(filtered_horizontal_lines, linediff_hor)
chess_lines_ver = get_chess_lines(filtered_vertical_lines, linediff_ver)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
img = cv2.imread(image_path)
img = cv2.resize(img, (1000, 1000))
for line in chess_lines_hor:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)
    
for line in chess_lines_ver:
    rho = line[0]
    theta = line[1]
    a = np.cos(theta)
    b = np.sin(theta)
    x0 = a*rho
    y0 = b*rho
    x1 = int(x0 + 5000*(-b))
    y1 = int(y0 + 5000*(a))
    x2 = int(x0 - 5000*(-b))
    y2 = int(y0 - 5000*(a))

    cv2.line(img,(x1,y1),(x2,y2),(0,0,255),2)

print(chess_lines_hor)
    
display_image(img)

In [ ]:
#Get equations of lines:
def getIntersection(rho1,theta1,rho2, theta2):
    A = np.array([
        [np.cos(theta1), np.sin(theta1)],
        [np.cos(theta2), np.sin(theta2)]
    ])
    b = np.array([[rho1], [rho2]])
    x0, y0 = np.linalg.solve(A, b)
    x0, y0 = int(np.round(x0)), int(np.round(y0))
    return [[x0, y0]]
    

In [ ]:
#Get all intersection points of horizontal and vertical lines to get the corners of the chess squares to extract patches. 

points = []
for i in chess_lines_hor:
    for j in chess_lines_ver:
        rho_hor = i[0]
        theta_hor = i[1]
        rho_ver = j[0]
        theta_ver = j[1]
        points.append(getIntersection(rho_hor, theta_hor, rho_ver, theta_ver))

In [ ]:
#points.insert(0, [0,points[0][0][1]])
previous = -1
width = img.shape[1]
for index, point in enumerate(points):
    if point[0][1] != previous:
        points.insert(index, [[width,previous]])
        previous = point[0][1]

        
previous = -1

for index, point in enumerate(points):
    if point[0][1] != previous:
        points.insert(index, [[0,point[0][1]]])
        previous = point[0][1]

points.pop(0)  
points.append([[width,points[-1][0][1] ]])
#print(points)

In [ ]:
# Make empty black image
img = cv2.imread(image_path)
img = cv2.resize(img, (1000, 1000))
for point in points:
    print(point)
    cv2.circle(img, (point[0][0],point[0][1]), 5, [0,0,255], thickness=2, lineType=8, shift=0)
#image[10,5]=[0,0,255]

display_image(img)
# # Make one pixel red

# # Save
# cv2.imwrite("result.png",image)